In [581]:
import csv 
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import binom
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
from math import factorial as fac
from lmfit import minimize, Parameters, fit_report

import matplotlib.pyplot as plot
import matplotlib.patches as mpatches

In [582]:
df = pd.read_csv('TL3_t5MTMMinformed6_noRem_withDups_frag_MTnorRNA_TcountANDTCconv_TL3_0m_MT.txt', sep='\t',header=(0)) 


In [583]:
n=len(df['Tcount'])
k=len(df.columns)
data=np.zeros((n,k-1))

for n in range(len(df['Tcount'])):
      for k in range(1,len(df.columns)):
        data[n,k-1]= df.iloc[n,k]

In [584]:
def input_TC(params):
    pe1 = params['Error_1']
    
    n=len(df['Tcount'])
    k=len(df.columns)
    model_res=np.zeros((n,k-1))
    
    def gs_func(x,y):
        return binom.pmf(y,x,pe1)
    
    for x in range(1,len(df['Tcount'])):
        for y in range(len(df.columns)-1):
            if x>=y:
                model_res[x-1,y]= gs_func(x,y)
                model_res[x-1,y] = model_res[x-1,y] * sum(data[x-1,])
                   
    return model_res


params = Parameters()
params.add('Error_1', value=0.00005, min=0, max=1)

def Two_TC_background(params):
    pe1 = params['Error_1']
    pe2 = params['Error_2']
    pi = params['frac_err']
    
    n=len(df['Tcount'])
    k=len(df.columns)
    model_Two=np.zeros((n,k-1))
    
    def gs_func(x,y):
        return pi*(binom.pmf(y,x,pe1)) + (1-pi)*(binom.pmf(y,x,pe2))
    
    for x in range(1,len(df['Tcount'])):
        for y in range(len(df.columns)-1):
            if x>=y:
                model_Two[x-1,y] = gs_func(x,y) * sum(data[x-1,])

    return model_Two

In [585]:
params = Parameters()
params.add('Error_1', value=0.0001, min=0, max=1)


def residual_TC1(params):
    return np.concatenate((input_TC(params) - (data)), axis=None)

out1 = minimize(residual_TC1, params, method='leastsquare')
out1

In [586]:
params = Parameters()
params.add('Error_1', value=0.01, min=0, max=1)
params.add('Error_2', value=0.0001, min=0, max=1)
params.add('frac_err', value=0.05, min=0, max=1)

def residual_TC2(params):
    return np.concatenate((Two_TC_background(params) - (data)), axis=None)

out2 = minimize(residual_TC2, params, method='leastsquare')
out2

In [587]:
def _aic1(n,k,rss):
    """first equation on page e7 of McShane et al
       k : number of parameters in model"""
    aic = 2.0*k + n*np.log(rss/n)
    return aic

#+ 2.0*k*(k+1)/(n-k-1) (As I was reading AIC under the Framework of Least Squares Estimation, it suggests to use this 
#correction parameter when n/K < 40 , so Iw asnt sure.Also suggests different AIC models for OLS optimization )
 
def calc_aic(n, rss1, rss2,k1,k2):
    """Calculate Akaike Information Criterium (AIC)
       from residuals for 1 and 2 state models
       n : number of data points
       rss1 : residual sum of squares of model 1
       rss2 : as rss1 for model 2
       k1 : number of free parameters of model 1
       k2 : as k1 for model 2
       """
    aic1 = _aic1(n, k1, rss1) 
    aic2 = _aic1(n, k2, rss2)
    aic_min = min(aic1, aic2)
    sum_of_probs = np.exp((aic_min - aic1) / 2) + np.exp((aic_min - aic2) / 2)
    prob_aic1 = np.exp((aic_min - aic1) / 2) / sum_of_probs
    prob_aic2 = np.exp((aic_min - aic2) / 2) / sum_of_probs
    
    print((aic_min - aic1), (aic_min - aic2))
    
    return [prob_aic1, prob_aic2]

calc_aic(out1.ndata,out1.chisqr,out2.chisqr,out1.nvarys,out2.nvarys)

-28.057368904843543 0.0


[8.080149626730492e-07, 0.9999991919850374]

In [588]:
df_Raw = pd.read_csv('TL3_t5MTMMinformed6_noRem_withDups_frag_MTnorRNA_TcountANDTCconv_TL3_60m_MT.txt',sep='\t',header=(0)) 
df_Raw

,Tcount,0,1,2,3,4,5,6
0,1,17,0,0,0,0,0,0
1,2,36,0,0,0,0,0,0
2,3,97,0,0,0,0,0,0
3,4,123,1,0,0,0,0,0
4,5,244,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
119,120,0,1,0,0,0,0,0
120,121,1,1,0,0,0,0,0
121,122,0,0,1,0,0,0,0
122,129,2,0,0,0,0,0,0


In [589]:
n=len(df_Raw['Tcount'])
k=len(df_Raw.columns)
Raw=np.zeros((n,k-1))

for x in range(0,len(df_Raw['Tcount'])):
      for y in range(0,len(df_Raw.columns)):
        Raw[x,y-1]= df_Raw.iloc[x,y]

In [590]:
def Two_TC(params):
    pc = params['TC']
    pe1 = 0.01725247
    pe2 = 5.7625e-04
    pi_err = 0.00409338
    pi = params['frac']
    
    n=len(df_Raw['Tcount'])
    k=len(df_Raw.columns)
    model_Two=np.zeros((n,k-1))
    
    def gs_func(x,y):
        return pi*(binom.pmf(y,x,pc)) + (1-pi)*(pi_err*binom.pmf(y,x,pe1)+(1-pi_err)*binom.pmf(y,x,pe2))
    
    for x in range(1,len(df_Raw['Tcount'])):
        for y in range(len(df_Raw.columns)-1):
            if x>=y:
                model_Two[x-1,y] = gs_func(x,y) * sum(Raw[x-1,])

    return model_Two


params = Parameters()
params.add('TC', value=0.001, min=0, max=1)
params.add('frac', value=0.01, min=0, max=1)

In [591]:
def Two_residual_TC(params):
    return np.concatenate((Two_TC(params) - (Raw)), axis=None)

In [592]:
out2 = minimize(Two_residual_TC, params, method='leastsquare')
out2